Анализ группы Вконтакте "Добрая работа".

In [36]:
import pandas as pd
import numpy as np
import requests # Импортируем модуль requests
from pprint import pprint # Импортируем функцию pprint()
import time # Импортируем модуль time
import plotly.express as px
token = 'd07d37d0d07d37d0d07d37d04fd0067b86dd07dd07d37d0b27667e9d71ac62943869309' # Указываем свой сервисный токен
dr_group_id = 'dobraya_rabota'

In [2]:
url = 'https://api.vk.com/method/groups.getMembers' # Указываем адрес обращения
params = {'group_id': dr_group_id, 'v': 5.95, 'access_token': token} # Формируем строку параметров
response = requests.get(url, params = params) # Посылаем запрос
max_count = response.json()['response']['count'] 
print('Количество пользователей: ', max_count)

Количество пользователей:  29292


In [3]:
count = 1000 
offset = 0 
dr_user_ids = [] 

while offset < max_count: 
    # Будем выгружать по count=5 пользователей, 
    # начиная с того места, где закончили на предыдущей итерации (offset) 
    print('Loading {} users, offset = {}'.format(count, offset))   
    params = {'group_id': dr_group_id, 'v': 5.95, 'count': count, 'offset': offset, 'access_token': token} 
    response = requests.get(url, params = params) 
    data = response.json() 
    dr_user_ids += data['response']['items'] 
    # Увеличиваем смещение на количество строк, которое мы уже выгрузили 
    offset += count 
#print(dr_user_ids) 

Loading 1000 users, offset = 0
Loading 1000 users, offset = 1000
Loading 1000 users, offset = 2000
Loading 1000 users, offset = 3000
Loading 1000 users, offset = 4000
Loading 1000 users, offset = 5000
Loading 1000 users, offset = 6000
Loading 1000 users, offset = 7000
Loading 1000 users, offset = 8000
Loading 1000 users, offset = 9000
Loading 1000 users, offset = 10000
Loading 1000 users, offset = 11000
Loading 1000 users, offset = 12000
Loading 1000 users, offset = 13000
Loading 1000 users, offset = 14000
Loading 1000 users, offset = 15000
Loading 1000 users, offset = 16000
Loading 1000 users, offset = 17000
Loading 1000 users, offset = 18000
Loading 1000 users, offset = 19000
Loading 1000 users, offset = 20000
Loading 1000 users, offset = 21000
Loading 1000 users, offset = 22000
Loading 1000 users, offset = 23000
Loading 1000 users, offset = 24000
Loading 1000 users, offset = 25000
Loading 1000 users, offset = 26000
Loading 1000 users, offset = 27000
Loading 1000 users, offset = 2800

In [4]:
# Create dictionary for users data
dr_users_data = {
    'id': [], 'last_name': [], 'first_name': [], 
    'sex': [], 'bdate': [], 'city': [], 'country': [],
    'home_town': [], 'has_mobile': [], 
    'universities': []
    }
url = 'https://api.vk.com/method/users.get' 

fields = list(map(str, dr_users_data.keys()))
print(fields)
flds = ",".join(map(str, fields[3:])) 
print(flds)

['id', 'last_name', 'first_name', 'sex', 'bdate', 'city', 'country', 'home_town', 'has_mobile', 'universities']
sex,bdate,city,country,home_town,has_mobile,universities


In [5]:
def get_optional_field(data, field):
    
    if field in data.keys():
        if field in ['city', 'country']:
            optional_field = data[field]['title'] + ', ' + str(data[field]['id'])
        else:
            optional_field = data[field]
            if field == 'education':
                print(optional_field)
    else:
        optional_field = np.nan
    
    return optional_field

In [ ]:
while len(dr_user_ids):
    part_ids = dr_user_ids[:1000]
    dr_user_ids = dr_user_ids[1000:]
    ids = ",".join(map(str, part_ids)) 
  
    params = {'user_id': ids, 'v': 5.95, 'fields': flds, 'access_token': token, 'lang': 'ru'}
    response = requests.get(url, params=params) # Send request
    #print(response.text) 
    users = response.json()['response'] # Extract user's data
    
    for user in users:
        #print(user)
        for field in fields:
            dr_users_data[field].append(get_optional_field(user, field))
 
    print('Waiting 0.5 sec...') 
    time.sleep(0.5) 
    
print('Requesting have finished') 

In [7]:
users_df = pd.DataFrame(dr_users_data)
#users_df.columns = ['ID', 'Фамилия', 'Имя', 'Пол', 'Дата рождения', '', '', '']

In [8]:
display(users_df.head())

,id,last_name,first_name,sex,bdate,city,country,home_town,has_mobile,universities
0,1509,Балберов,Кирилл,2,11.9.1986,"Санкт-Петербург, 2","Россия, 1",NaN,1.0,NaN
1,1558,Колкова,Ольга,1,26.11.1984,"Санкт-Петербург, 2","Россия, 1",NaN,1.0,NaN
2,1725,Чижова,Анастасия,1,NaN,"Санкт-Петербург, 2","Россия, 1",NaN,1.0,NaN
3,1947,Егорова,Екатерина,1,25.6,"Санкт-Петербург, 2","Россия, 1",Санкт-Петербург,1.0,"[{'chair': 1764773, 'chair_name': 'Системного ..."
4,2835,Слав,Евгения,1,13.4,"Санкт-Петербург, 2","Россия, 1",Санкт-Петербург,1.0,"[{'chair': 221, 'chair_name': 'Прикладной инфо..."


In [9]:
users_df.to_csv('data/dobraya_rabota_users.csv')

In [10]:
def get_gender_feature(sex):
    if sex == 1:
        return 'Female'
    elif sex == 2:
        return 'Male'
    elif sex == 0 or sex == np.nan:
        return 'Other'

In [ ]:
users_df['sex'].apply(get_gender_feature)

In [12]:
def get_age_feature(age):
    age = str(age)
    if (age == np.nan) or (len(age) < 6):
        return 0
    else:
        age = 2022 - int(age.split('.')[2])
        return age

In [13]:
users_df['age'] = users_df['bdate'].apply(get_age_feature)

In [30]:
def get_zodiac_sign_feature(bdate):
    
    date = str(bdate).split('.')[:2]
    print(date)
    if date[0] == 'nan':
        return 'Не указано'
    month = int(date[1])
    day = int(date[0])
    if (month == 12 and day >= 22) or (month == 1 and day <= 19):
        return 'Козерог'
    elif (month == 1 and day >= 20) or (month == 2 and day <= 18):
        return 'Водолей'
    elif (month == 2 and day >= 19) or (month == 3 and day <= 20):
        return 'Рыбы'
    elif (month == 3 and day >= 21) or (month == 4 and day <= 19):
        return 'Овен'
    elif (month == 4 and day >= 20) or (month == 5 and day <= 20):
        return 'Телец'
    elif (month == 5 and day >= 21) or (month == 6 and day <= 20):
        return 'Близнецы'
    elif (month == 6 and day >= 21) or (month == 7 and day <= 22):
        return 'Рак'
    elif (month == 7 and day >= 23) or (month == 8 and day <= 22):
        return 'Лев'
    elif (month == 8 and day >= 23) or (month == 9 and day <= 22):
        return 'Дева'
    elif (month == 9 and day >= 23) or (month == 10 and day <= 22):
        return 'Весы'
    elif (month == 10 and day >= 23) or (month == 11 and day <= 21):
        return 'Скорпион'
    elif (month == 11 and day >= 22) or (month == 12 and day <= 21):
        return 'Стрелец' 

In [ ]:
users_df['zodiac_sign'] = users_df['bdate'].apply(get_zodiac_sign_feature)

In [40]:
# Draw the hyistogram
fig = px.histogram(
    data_frame=users_df[users_df['zodiac_sign'] != 'Не указано'], 
    #nbins=1200,
    x='zodiac_sign',
    orientation='v', 
    width=850, height=350, 
    title='Distribution of zodiac signs (histogram)' 
);
fig.show()

In [32]:
display(users_df.head())

,id,last_name,first_name,sex,bdate,city,country,home_town,has_mobile,universities,age,zodiac_sign
0,1509,Балберов,Кирилл,2,11.9.1986,"Санкт-Петербург, 2","Россия, 1",NaN,1.0,NaN,36,Дева
1,1558,Колкова,Ольга,1,26.11.1984,"Санкт-Петербург, 2","Россия, 1",NaN,1.0,NaN,38,Стрелец
2,1725,Чижова,Анастасия,1,NaN,"Санкт-Петербург, 2","Россия, 1",NaN,1.0,NaN,0,Не указано
3,1947,Егорова,Екатерина,1,25.6,"Санкт-Петербург, 2","Россия, 1",Санкт-Петербург,1.0,"[{'chair': 1764773, 'chair_name': 'Системного ...",0,Рак
4,2835,Слав,Евгения,1,13.4,"Санкт-Петербург, 2","Россия, 1",Санкт-Петербург,1.0,"[{'chair': 221, 'chair_name': 'Прикладной инфо...",0,Овен


In [ ]:
users_df['first_name'].value_counts().sort_values(ascending=False)[:20]

In [137]:
users_df[users_df['age'] > 0]['age'].mean()

35.517209961530675

In [43]:
# Draw the hyistogram
fig = px.histogram(
    data_frame=users_df[users_df['age'] > 0]['age'], 
    nbins=100,
    x='age',
    orientation='v', 
    width=850, height=350, 
    title='Distribution of users age' 
);
fig.show()

In [108]:
display(users_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29280 entries, 0 to 29279
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            29280 non-null  int64  
 1   last_name     29280 non-null  object 
 2   first_name    29280 non-null  object 
 3   sex           29280 non-null  object 
 4   bdate         21537 non-null  object 
 5   city          22419 non-null  object 
 6   country       25430 non-null  object 
 7   home_town     9780 non-null   object 
 8   has_mobile    29241 non-null  float64
 9   universities  9780 non-null   object 
dtypes: float64(1), int64(1), object(8)
memory usage: 2.2+ MB


None